In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np
from sklearn.linear_model import LinearRegression

# Part 1: Fetch Data from Excel Sheets
def fetch_data(file_path, sheet_name):
    # Load the user data from Excel
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    if 'startDate' in data.columns and 'endDate' in data.columns:
        data['startDate'] = pd.to_datetime(data['startDate'])
        data['endDate'] = pd.to_datetime(data['endDate'])
    elif 'date' in data.columns:
        data['date'] = pd.to_datetime(data['date'])
    else:
        return data
    return data


In [2]:
# Part 2: HRV Recovery Score Calculation
def calculate_hrv_recovery_score(merged_data, baseline_sdnn, baseline_rmssd):
# def calculate_hrv_recovery_score(hrv_data, baseline_sdnn, baseline_rmssd):
    hrv_metrics = {}
    ideal_lf_hf_ratio = 1
    w_sdnn,w_rmssd = 0.4,0.4

    for date in merged_data['startDate'].unique():
        current_sdnn = merged_data[merged_data['startDate'] == date]['SDNN (ms)'].values[0]
        current_rmssd = merged_data[merged_data['startDate'] == date]['RMSSD (ms)'].values[0]
        current_lf_hf_ratio = merged_data[merged_data['startDate'] == date]['LF/HF Ratio'].values[0]

        # Calculate components relative to baseline
        sdnn_component = current_sdnn / baseline_sdnn if baseline_sdnn > 0 else 0
        rmssd_component = current_rmssd / baseline_rmssd if baseline_rmssd > 0 else 0
        lf_hf_component = ideal_lf_hf_ratio / current_lf_hf_ratio if current_lf_hf_ratio > 0 else 0

        # HRV Recovery Score with proposed weights
        hrv_recovery_score = w_sdnn * sdnn_component + w_rmssd * rmssd_component + (1-(w_sdnn+w_rmssd)) * lf_hf_component
        # hrv_recovery_score = 0.4 * sdnn_component + 0.4 * rmssd_component

        hrv_metrics[date] = hrv_recovery_score

    return hrv_metrics

In [3]:
# Load the Excel file and the specific sheet
file_path = r'C:\Users\amank\Downloads\Fithack\Recovery\vitality_extract_data.xlsx'  # Adjust the path if needed
sheet_name = 'V_HR_Agg'  # The sheet containing RHR data

# Read the Excel sheet into a pandas DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Convert the date column to datetime if it's not already
df['date'] = pd.to_datetime(df['date'])

# Sort data by date for easier indexing
df = df.sort_values(by='date')

print(df.columns)

Index(['userName', 'valueGeneratedAt', 's_name', 'date', 'type', 'unit',
       'valueType', 'value'],
      dtype='object')


In [4]:
# Load the Excel file and the specific sheet
file_path = r'C:\Users\amank\Downloads\Fithack\Recovery\vitality_extract_data.xlsx'  # Adjust the path if needed
sheet_name = 'V_HR_Agg'  # The sheet containing RHR data

# Read the Excel sheet into a pandas DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Strip any leading/trailing spaces from the column names
df.columns = df.columns.str.strip()

# Convert the date column to datetime if it's not already
df['date'] = pd.to_datetime(df['date'])

# Check the structure of the DataFrame
print(df.head())

# Create the pivot table
pivot_table = df.pivot_table(
    index='date',            # Rows will be the date
    columns='valueType',    # Columns will be the value type
    values='value',         # Values to aggregate
    aggfunc='sum',          # Aggregation function
    fill_value=0            # Fill missing values with 0
).reset_index()

# Display the pivot table with the restingAvg column
print(pivot_table)

# Save the pivot table to a new Excel file
output_file_path = r'C:\Users\amank\Downloads\Fithack\Recovery\V_HR_Agg_pivot_table.xlsx'
pivot_table.to_excel(output_file_path, index=False)
print(f"Pivot table saved to {output_file_path}")

                               userName     valueGeneratedAt s_name  \
0  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  2024-10-02 20:53:43   V_HR   
1  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  2024-10-02 20:53:43   V_HR   
2  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  2024-10-02 20:53:43   V_HR   
3  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  2024-10-02 20:53:43   V_HR   
4  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  2024-10-02 20:53:43   V_HR   

        date                               type       unit    valueType  value  
0 2024-09-19  HKQuantityTypeIdentifierHeartRate  count/min       dayAvg   79.6  
1 2024-09-19  HKQuantityTypeIdentifierHeartRate  count/min       dayMin   49.0  
2 2024-09-19  HKQuantityTypeIdentifierHeartRate  count/min       dayMax  136.0  
3 2024-09-19  HKQuantityTypeIdentifierHeartRate  count/min  activityAvg  100.6  
4 2024-09-19  HKQuantityTypeIdentifierHeartRate  count/min  activityMin   59.0  
valueType       date  activityAvg  activityMax  activityMin  dayAvg  dayMax  \
0       

In [5]:
# Path to the pivot table Excel file
pivot_file_path = r'C:\Users\amank\Downloads\Fithack\Recovery\V_HR_Agg_pivot_table.xlsx'  # Adjust the path as necessary

# Load the pivot table Excel sheet into a DataFrame
pivot_df = pd.read_excel(pivot_file_path)

# Display the first few rows of the DataFrame
print(pivot_df.head())

        date  activityAvg  activityMax  activityMin  dayAvg  dayMax  dayMin  \
0 2024-06-21        101.1          124         60.0    78.6     130    47.0   
1 2024-06-22         95.1          128         58.7    76.4     128    48.0   
2 2024-06-23         95.5          122         68.0   101.9     155    53.0   
3 2024-06-24        102.3          127         44.0    99.3     127    44.0   
4 2024-06-25         96.0          132         55.0    83.4     132    47.0   

   restingAvg  restingMax  restingMin  sleepAvg  sleepMax  sleepMin  \
0        77.6       130.0        50.5      58.4      84.0      47.0   
1        80.2       112.0        51.0      53.5      59.0      48.0   
2        81.6       115.0        53.0       0.0       0.0       0.0   
3        93.0       106.0        69.0       0.0       0.0       0.0   
4        86.9       125.0        53.0      59.4      84.0      47.0   

   workoutAvg  workoutMax  workoutMin  
0         0.0           0           0  
1         0.0     

In [6]:
# Function to calculate baseline RHR for each day based on the last 90 days or the minimum available date
def calculate_baseline_rhr(df):
    baseline_rhr = []

    for idx, row in df.iterrows():
        current_date = row['date']

        # Rolling 90-day Window: For each date, the script looks back 90 days or uses the minimum available date, whichever is more recent.
        baseline_start = max(df['date'].min(), current_date - timedelta(days=90))

        # Filter the data to get the baseline period
        baseline_period = df[(df['date'] >= baseline_start) & (df['date'] <= current_date)]

        # Calculate the average resting heart rate during the baseline period
        avg_rhr = baseline_period['restingAvg'].mean()

        # Append the baseline RHR for the current date
        baseline_rhr.append(avg_rhr)

    # Create a new column in the DataFrame for the baseline RHR
    df['baseline_rhr'] = baseline_rhr

    return df

# Apply the function to the DataFrame
df_with_baseline_rhr = calculate_baseline_rhr(pivot_df)

# Display the first few rows of the DataFrame with baseline RHR
print(df_with_baseline_rhr[['date', 'restingAvg', 'baseline_rhr']])

# Save the output to a new Excel file if needed
output_file_path = r'C:\Users\amank\Downloads\Fithack\Recovery\baseline_rhr_output.xlsx'
df_with_baseline_rhr.to_excel(output_file_path, index=False)
print(f"Baseline RHR data saved to {output_file_path}")

         date  restingAvg  baseline_rhr
0  2024-06-21        77.6     77.600000
1  2024-06-22        80.2     78.900000
2  2024-06-23        81.6     79.800000
3  2024-06-24        93.0     83.100000
4  2024-06-25        86.9     83.860000
..        ...         ...           ...
86 2024-09-15        82.4     81.134483
87 2024-09-16        81.2     81.135227
88 2024-09-17        80.2     81.124719
89 2024-09-18        79.7     81.108889
90 2024-09-19        80.0     81.096703

[91 rows x 3 columns]
Baseline RHR data saved to C:\Users\amank\Downloads\Fithack\Recovery\baseline_rhr_output.xlsx


In [7]:
def calculate_rhr_recovery_score(rhr_data, baseline_rhr):
    rhr_metrics = {}

    for date in rhr_data['date'].unique():
        current_rhr = rhr_data[rhr_data['date'] == date]['restingAvg'].values[0]
        rhr_recovery_score = (baseline_rhr - current_rhr) / baseline_rhr

        rhr_metrics[date] = rhr_recovery_score

    return rhr_metrics

In [8]:
# Part 4: Sleep Recovery Score Calculation
def calculate_sleep_recovery_score(sleep_data):
    sleep_metrics = {}
    w_ts, w_ds, w_rs, w_sqi = 0.30, 0.25, 0.20, 0.25
    w_se, w_sol, w_waso, w_sqr = 0.4, 0.2, 0.2, 0.2

    for date in sleep_data['date'].unique():
        total_sleep = sleep_data[sleep_data['date'] == date]['TotalSleepDuration'].values[0]
        deep_sleep = sleep_data[sleep_data['date'] == date]['TotalDeepSleepDuration'].values[0]
        rem_sleep = sleep_data[sleep_data['date'] == date]['TotalREMSleepDuration'].values[0]

        # Calculate sleep efficiency, onset latency (SOL), wake after sleep onset (WASO)
        total_in_bed_duration = sleep_data[sleep_data['date'] == date]['TotalInBedDuration'].values[0]
        sleep_efficiency = (total_sleep / total_in_bed_duration) * 100
        sol_time = sleep_data[sleep_data['date'] == date]['SOL'].values[0]
        waso_time = sleep_data[sleep_data['date'] == date]['WASO'].values[0]

        # SOL and WASO score
        sol_score = 100 - (sol_time / 10)
        waso_score = 100 - (waso_time / 10)

        if 'SleepQualityRating' in sleep_data.columns:
            sleep_quality_rating = sleep_data[sleep_data['date'] == date]['SleepQualityRating'].values[0]
            sqi = w_se * sleep_efficiency + w_sol * sol_score + w_waso * waso_score + w_sqr * sleep_quality_rating
        else:
            sqi = w_se * sleep_efficiency + w_sol * sol_score + w_waso * waso_score
        


        # Sleep Quality Index (SQI)
        

        # Final Sleep Recovery Score
        sleep_recovery_score = (
            w_ts * total_sleep +
            w_ds * deep_sleep +
            w_rs * rem_sleep +
            w_sqi * sqi
        )

        sleep_metrics[date] = sleep_recovery_score

    return sleep_metrics

In [9]:
# Part 5: Wellness Recovery Score Calculation
def calculate_wellness_recovery_score(wellness_data):
    wellness_metrics = {}

    for date in wellness_data['date'].unique():
        mood = wellness_data[wellness_data['date'] == date]['Mood'].values[0]
        fatigue = wellness_data[wellness_data['date'] == date]['Fatigue'].values[0]
        soreness = wellness_data[wellness_data['date'] == date]['Soreness'].values[0]
        stress = wellness_data[wellness_data['date'] == date]['Stress'].values[0]

        # Wellness Recovery Score
        wellness_recovery_score = 0.30 * mood + 0.25 * fatigue + 0.20 * soreness + 0.25 * stress

        wellness_metrics[date] = wellness_recovery_score

    return wellness_metrics

In [26]:
# Create dummy wellness data
dummy_wellness_data = pd.DataFrame({
    'date': pd.date_range(start='2024-09-01', periods=10, freq='D'),
    'Mood': np.random.randint(1, 10, size=10),
    'Fatigue': np.random.randint(1, 10, size=10),
    'Soreness': np.random.randint(1, 10, size=10),
    'Stress': np.random.randint(1, 10, size=10)
})

# Display the dummy wellness data
# print(dummy_wellness_data)

# Calculate wellness recovery score using the dummy data
dummy_wellness_recovery_score = calculate_wellness_recovery_score(dummy_wellness_data)

# Display the dummy wellness recovery score
print(dummy_wellness_recovery_score)

{Timestamp('2024-09-01 00:00:00'): 6.45, Timestamp('2024-09-02 00:00:00'): 4.300000000000001, Timestamp('2024-09-03 00:00:00'): 3.3, Timestamp('2024-09-04 00:00:00'): 5.6499999999999995, Timestamp('2024-09-05 00:00:00'): 3.7, Timestamp('2024-09-06 00:00:00'): 5.7, Timestamp('2024-09-07 00:00:00'): 5.45, Timestamp('2024-09-08 00:00:00'): 7.300000000000001, Timestamp('2024-09-09 00:00:00'): 4.75, Timestamp('2024-09-10 00:00:00'): 3.25}


In [11]:
# Part 6: Training Load Modulation Calculation
def calculate_training_load_modulation(load_data, strain_data):
    training_load_metrics = {}
    w_cr = 0.5
    for date in load_data['date'].unique():
        current_load = load_data[load_data['date'] == date]['CurrentLoad'].values[0]
        strain_score = strain_data[strain_data['date'] == date]['StrainScore'].values[0]

        # Training Load Modulation
        training_load_modulation = w_cr * current_load + 1-w_cr * strain_score

        training_load_metrics[date] = training_load_modulation

    return training_load_metrics

In [54]:
dummy_training_load_modulation = {}
dummy_wellness_recovery_score = {}

In [12]:
# Function to calculate baseline HRV for each day based on the last 90 days or the minimum available date
def calculate_baseline_hrv(df):
    baseline_hrv = []
    df['creationDate'] = pd.to_datetime(df['creationDate']).dt.date
    for idx, row in df.iterrows():
        current_date = row['creationDate']

        # Rolling 90-day Window: For each date, the script looks back 90 days or uses the minimum available date, whichever is more recent.
        baseline_start = max(df['creationDate'].min(), current_date - timedelta(days=90))

        # Filter the data to get the baseline period
        baseline_period = df[(df['creationDate'] >= baseline_start) & (df['creationDate'] <= current_date)]

        # Calculate the average RMSSD during the baseline period
        avg_hrv = baseline_period['RMSSD (ms)'].mean()

        # Append the baseline HRV for the current date
        baseline_hrv.append(avg_hrv)

    # Create a new column in the DataFrame for the baseline HRV
    df['baseline_hrv'] = baseline_hrv
    df['date'] = df['creationDate']

    return df


file_path_prophrv = r'C:\Users\amank\Downloads\Fithack\Recovery\PropHRV.xlsx'
df = fetch_data(file_path_prophrv, 'VPropHRV')

# Apply the function to the DataFrame
df_with_baseline_hrv = calculate_baseline_hrv(df)

# Display the first few rows of the DataFrame with baseline HRV
print(df_with_baseline_hrv[['date', 'RMSSD (ms)', 'baseline_hrv']])

# Save the output to a new Excel file if needed
output_file_path = r'C:\Users\amank\Downloads\Fithack\Recovery\baseline_hrv_output.xlsx'
df_with_baseline_hrv.to_excel(output_file_path, index=False)
print(f"Baseline HRV data saved to {output_file_path}")


           date  RMSSD (ms)  baseline_hrv
0    2024-09-19       80.86     59.646028
1    2024-09-19       85.16     59.646028
2    2024-09-19      132.21     59.646028
3    2024-09-19      108.03     59.646028
4    2024-09-19       86.70     59.646028
..          ...         ...           ...
909  2024-06-21       24.93     79.571818
910  2024-06-21      127.72     79.571818
911  2024-06-21      180.76     79.571818
912  2024-06-21       36.53     79.571818
913  2024-06-21       24.99     79.571818

[914 rows x 3 columns]
Baseline HRV data saved to C:\Users\amank\Downloads\Fithack\Recovery\baseline_hrv_output.xlsx


In [37]:
# Part 7: HRV Modulation Adjustment Calculation (Linear Regression)
def calculate_hrv_modulation_adjustment(hrv_data, baseline_hrv):
    hrv_modulation_metrics = {}

    for date in hrv_data['date'].unique():
        recent_trend_data = hrv_data[hrv_data['date'] <= date].tail(7)  # last 7 days
        if len(recent_trend_data) >= 2:
            # X = day index, Y = HRV values
            X = np.array(range(len(recent_trend_data))).reshape(-1, 1)
            Y = recent_trend_data['RMSSD (ms)'].values.reshape(-1, 1)
            model = LinearRegression().fit(X, Y)
            recent_trend_slope = model.coef_[0][0]
        else:
            recent_trend_slope = 0

        hrv_modulation_adjustment = (recent_trend_slope - baseline_hrv) / baseline_hrv if baseline_hrv > 0 else 0

        hrv_modulation_metrics[date] = hrv_modulation_adjustment

    return hrv_modulation_metrics

In [14]:
# Part 8: Comprehensive Recovery Score Calculation with Dynamic Weight Adjustment
def calculate_comprehensive_recovery_score(hrv_score, rhr_score, sleep_score, wellness_score, training_modulation, hrv_modulation):
    recovery_scores = {}

    for date in hrv_score.keys():
        # Prepare a list of available factors and their corresponding weights
        available_factors = []
        available_weights = []

        # Add factors and weights only if the factor is available for the current date
        if date in hrv_score:
            available_factors.append(hrv_score[date])
            available_weights.append(0.4)

        if date in rhr_score:
            available_factors.append(rhr_score[date])
            available_weights.append(0.3)

        if date in sleep_score:
            available_factors.append(sleep_score[date])
            available_weights.append(0.3)

        if date in wellness_score:
            available_factors.append(wellness_score[date])
            available_weights.append(0.2)

        if date in training_modulation:
            available_factors.append(training_modulation[date])
            available_weights.append(0.1)

        if date in hrv_modulation:
            available_factors.append(hrv_modulation[date])
            available_weights.append(0.1)

        # Recalculate the weights so they sum up to 1 (100%)
        total_weight = sum(available_weights)
        normalized_weights = [w / total_weight for w in available_weights]

        # Calculate the comprehensive recovery score using the normalized weights
        comprehensive_recovery_score = sum(factor * weight for factor, weight in zip(available_factors, normalized_weights))

        # Store the result for this date
        recovery_scores[date] = comprehensive_recovery_score

    return recovery_scores

In [47]:
# Sheet paths
file_path_hrv = r'C:\Users\amank\Downloads\Fithack\Recovery\vitality_extract_data.xlsx'
file_path_baseline_hrv = r'C:\Users\amank\Downloads\Fithack\Recovery\baseline_hrv_output.xlsx'
file_path_prophrv = r'C:\Users\amank\Downloads\Fithack\Recovery\PropHRV.xlsx'
file_path_rhr = r'C:\Users\amank\Downloads\Fithack\Recovery\baseline_rhr_output.xlsx'
file_path_sleep = r'C:\Users\amank\Downloads\Fithack\Recovery\sleep_extract.xlsx'

In [48]:
#calculate hrv recovery score and rhr recovery score
hrv_data = fetch_data(file_path_baseline_hrv, 'Sheet1')
lf_hf_data = fetch_data(file_path_prophrv, 'VPropHRVFrequency')
rhr_data = fetch_data(file_path_rhr, 'Sheet1')
merged_data = pd.merge(hrv_data, lf_hf_data[['startDate', 'LF/HF Ratio']], on='startDate', how='inner')

# Baselines
baseline_sdnn = merged_data['SDNN (ms)'].mean()
baseline_rmssd = merged_data['RMSSD (ms)'].mean()
baseline_rhr = rhr_data['baseline_rhr'].mean()
baseline_hrv = hrv_data['baseline_hrv'].mean()
hrv_recovery_score = calculate_hrv_recovery_score(merged_data, baseline_sdnn, baseline_rmssd)
rhr_recovery_score = calculate_rhr_recovery_score(rhr_data, baseline_rhr)

In [30]:
hrv_recovery_score_df = pd.DataFrame(list(hrv_recovery_score.items()), columns=['date', 'hrv_recovery_score'])
hrv_recovery_score_df['date'] = pd.to_datetime(hrv_recovery_score_df['date'])
hrv_recovery_score_df.to_excel(r'C:\Users\amank\Downloads\Fithack\Recovery\hrv_recovery_score.xlsx', index=False)

In [32]:
rhr_recovery_score_df = pd.DataFrame(list(rhr_recovery_score.items()), columns=['date', 'rhr_recovery_score'])
rhr_recovery_score_df['date'] = pd.to_datetime(rhr_recovery_score_df['date'])
rhr_recovery_score_df.to_excel(r'C:\Users\amank\Downloads\Fithack\Recovery\rhr_recovery_score.xlsx', index=False)

In [17]:
#sleep recovery score
sleep_data_pivot = fetch_data(file_path_sleep, 'Pivot Table')
header_row_values = [
    "Row Labels", "TotalAwakeDuration", "TotalCoreSleepDuration", 
    "TotalDeepSleepDuration", "TotalInBedDuration", 
    "TotalREMSleepDuration", "TotalSleepDuration", "(blank)", "Grand Total"
]
header_row_index = sleep_data_pivot[sleep_data_pivot.iloc[:, 0] == header_row_values[0]].index[0]
sleep_data_pivot.columns = sleep_data_pivot.iloc[header_row_index]
sleep_data_pivot = sleep_data_pivot.iloc[header_row_index + 1:].reset_index(drop=True)
sleep_data_waso = fetch_data(file_path_sleep, 'S_WASO_Agg')
sleep_data_sol = fetch_data(file_path_sleep, 'S_OnsetLatency_Agg')


sleep_data_pivot['date'] = pd.to_datetime(sleep_data_pivot['Row Labels'] + "-2024", format="%d-%b-%Y", errors='coerce')
sleep_data_pivot = sleep_data_pivot.sort_values(by='date', ascending=False)

sleep_data_sol.rename(columns={'value': 'SOL'}, inplace=True)
sleep_data_waso.rename(columns={'value': 'WASO'}, inplace=True)
merged_sleep_data = pd.merge(sleep_data_pivot, sleep_data_sol[sleep_data_sol['valueType'] == 'SleepOnsetLatency'][['date', 'SOL']], on = 'date', how="inner")
merged_sleep_data = pd.merge(merged_sleep_data, sleep_data_waso[['date','WASO']], on = 'date', how="inner")

sleep_recovery_score = calculate_sleep_recovery_score(merged_sleep_data)

In [34]:
sleep_recovery_score_df = pd.DataFrame(list(sleep_recovery_score.items()), columns=['date', 'sleep_recovery_score'])
sleep_recovery_score_df['date'] = pd.to_datetime(sleep_recovery_score_df['date'])
sleep_recovery_score_df.to_excel(r'C:\Users\amank\Downloads\Fithack\Recovery\sleep_recovery_score.xlsx', index=False)

In [51]:
# HRV Modulation Adjustment Calculation
hrv_modulation = calculate_hrv_modulation_adjustment(hrv_data, baseline_hrv)

In [53]:
print(hrv_modulation)

{Timestamp('2024-09-19 00:00:00'): -1.1547109908453395, Timestamp('2024-09-18 00:00:00'): -1.1547109908453395, Timestamp('2024-09-17 00:00:00'): -1.1547109908453395, Timestamp('2024-09-16 00:00:00'): -1.1547109908453395, Timestamp('2024-09-15 00:00:00'): -1.1547109908453395, Timestamp('2024-09-14 00:00:00'): -1.1547109908453395, Timestamp('2024-09-13 00:00:00'): -1.1547109908453395, Timestamp('2024-09-12 00:00:00'): -1.1547109908453395, Timestamp('2024-09-11 00:00:00'): -1.1547109908453395, Timestamp('2024-09-10 00:00:00'): -1.1547109908453395, Timestamp('2024-09-09 00:00:00'): -1.1547109908453395, Timestamp('2024-09-08 00:00:00'): -1.1547109908453395, Timestamp('2024-09-07 00:00:00'): -1.1547109908453395, Timestamp('2024-09-06 00:00:00'): -1.1547109908453395, Timestamp('2024-09-05 00:00:00'): -1.1547109908453395, Timestamp('2024-09-04 00:00:00'): -1.1547109908453395, Timestamp('2024-09-03 00:00:00'): -1.1547109908453395, Timestamp('2024-09-02 00:00:00'): -1.1547109908453395, Timestamp

In [59]:
hrv_modulation_df = pd.DataFrame(list(hrv_modulation.items()), columns=['date', 'hrv_modulation'])
hrv_modulation_df['date'] = pd.to_datetime(hrv_modulation_df['date'])
hrv_modulation_df.to_excel(r'C:\Users\amank\Downloads\Fithack\Recovery\hrv_modulation.xlsx', index=False)

In [58]:
comprehensive_recovery_score = calculate_comprehensive_recovery_score(hrv_recovery_score, rhr_recovery_score, sleep_recovery_score, dummy_wellness_recovery_score,dummy_training_load_modulation , hrv_modulation)
comprehensive_recovery_score_df = pd.DataFrame(list(comprehensive_recovery_score.items()), columns=['date', 'comprehensive_recovery_score'])
comprehensive_recovery_score_df['date'] = pd.to_datetime(comprehensive_recovery_score_df['date'])
comprehensive_recovery_score_df.to_excel(r'C:\Users\amank\Downloads\Fithack\Recovery\comprehensive_recovery_score.xlsx', index=False)

In [ ]:


# print(merged_sleep_data)
# wellness_data = fetch_data(file_path, 'WellnessData')
# load_data = fetch_data(file_path, 'LoadData')
# strain_data = fetch_data(file_path, 'StrainData')




# Calculate individual recovery scores



# wellness_recovery_score = calculate_wellness_recovery_score(wellness_data)
# training_load_modulation = calculate_training_load_modulation(load_data, strain_data)
# hrv_modulation_adjustment = calculate_hrv_modulation_adjustment(hrv_data, baseline_hrv)

# Calculate comprehensive recovery score
# comprehensive_recovery_score = calculate_comprehensive_recovery_score(
#     hrv_recovery_score, rhr_recovery_score, sleep_recovery_score, wellness_recovery_score,
#     training_load_modulation, hrv_modulation_adjustment
# )

# Output the final scores
# comprehensive_recovery_df = pd.DataFrame.from_dict(comprehensive_recovery_score, orient='index', columns=['Comprehensive Recovery Score'])
# print(comprehensive_recovery_df)